In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='o1-mini')
llm


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000026DAFBC9810>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026DB05D4150>, root_client=<openai.OpenAI object at 0x0000026DAF587F90>, root_async_client=<openai.AsyncOpenAI object at 0x0000026DB05C8650>, model_name='o1-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('autopsy_of_john_doe.txt')
loader

In [5]:
text_document = loader.load()
text_document

[Document(metadata={'source': 'autopsy_of_john_doe.txt'}, page_content=' "The Autopsy of Jane Doe" is a 2016 horror film directed by AndrÃ© Ã˜vredal.  It tells the story of a father and son, Tommy and Austin Tilden, who are coroners in a small town. One night, they receive the body of an unidentified woman (Jane Doe) for autopsy.  What follows is a night of increasingly bizarre and terrifying discoveries as they try to determine her cause of death.\n\nThe film relies heavily on atmosphere and suspense.  The confined setting of the autopsy room, the progressively revealed details of Jane Doe\'s body, and the growing sense of dread all contribute to a palpable tension.  The film excels at creating a claustrophobic and unsettling environment, making the viewer feel trapped alongside the Tildens.  However, this strength can also be a point of contention.  Does the film rely *too* heavily on atmosphere at the expense of a coherent plot?  Some might argue that the escalating supernatural eve

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splitter

In [11]:
document = splitter.split_documents(text_document)
document

[Document(metadata={'source': 'autopsy_of_john_doe.txt'}, page_content='"The Autopsy of Jane Doe" is a 2016 horror film directed by AndrÃ© Ã˜vredal.  It tells the story of a father and son, Tommy and Austin Tilden, who are coroners in a small town. One night, they receive the body of an unidentified woman (Jane Doe) for autopsy.  What follows is a night of increasingly bizarre and terrifying discoveries as they try to determine her cause of death.'),
 Document(metadata={'source': 'autopsy_of_john_doe.txt'}, page_content="The film relies heavily on atmosphere and suspense.  The confined setting of the autopsy room, the progressively revealed details of Jane Doe's body, and the growing sense of dread all contribute to a palpable tension.  The film excels at creating a claustrophobic and unsettling environment, making the viewer feel trapped alongside the Tildens.  However, this strength can also be a point of contention.  Does the film rely *too* heavily on atmosphere at the expense of a

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002')

vector_store = FAISS.from_documents(document, embeddings)

In [15]:
retriever = vector_store.as_retriever()

In [17]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
  input_variables=['context', 'question'],
  template='''
Answer the following question based on the given context:
<context>
{context}
</context>

Question: {question}
Answer:'''
)

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chain = create_stuff_documents_chain(
  prompt=prompt,
 llm=llm,
)

In [23]:
question = 'is this a horror movie or a family watch?'
context = retriever.get_relevant_documents(question)

response = chain.invoke({
  'context': context,
  'question': question
})

In [27]:
print(response)

"The Autopsy of Jane Doe" is decidedly a **horror movie** rather than a family-friendly film. Here's why:

1. **Atmosphere and Suspense**: The film heavily relies on creating a claustrophobic and unsettling environment within the autopsy room. This intense atmosphere is characteristic of horror genres aiming to evoke fear and tension.

2. **Supernatural Elements**: The escalating supernatural events surrounding Jane Doe introduce elements that are designed to scare and disturb the audience. These occurrences often lack logical progression, serving more to generate scares than to develop a coherent, family-appropriate narrative.

3. **Intense Climax**: The climax involves chaotic and visually disturbing sequences that are typical of horror films. Such scenes are meant to elicit strong emotional reactions, including fear and anxiety, which are not suitable for all audiences, especially younger viewers.

4. **Themes and Content**: The movie delves into dark themes, including death, the un